In [1]:
import os
import pandas as pd
import numpy as np
import sys
import csv
from time import strptime
import json
from datetime import datetime as dt
from datetime import date
from statistics import mean
import math
import pickle
import time

In [2]:
# csv.field_size_limit(sys.maxsize)
# np.set_printoptions(threshold=sys.maxsize)
pd.options.display.max_columns = None
pd.options.display.float_format = '{:20,.15f}'.format
# pd.options.display.max_rows = 10000
# pd.set_option('display.max_colwidth', -1)

In [ ]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [3]:
def calc_smooth_mean(df, by, on, m):
    mean = df[on].mean()
    agg = df.groupby(by)[on].agg(['count', 'mean'])
    counts = agg['count']
    means = agg['mean']
    smooth = (counts * means + m * mean) / (counts + m)
    return df[by].map(smooth)

In [4]:
df_file = pd.read_csv('savedcsv/17_homepage.csv')
df_file = df_file.drop(df_file[df_file['META__id'] == 6].index)
df_file.to_csv('savedcsv/df_file.csv', index=False)
df_file = pd.read_csv('savedcsv/df_file.csv')

In [5]:
# df_raw = reduce_mem_usage(pd.read_csv('savedcsv/17_homepage.csv'))
df_raw = pd.read_csv('savedcsv/17_homepage.csv')
df_raw['META__profitability'] = df_raw['profitability']
df_raw['META__year_avg_profitability'] = df_raw['year_avg_profitability']
df_raw = df_raw.drop(['profitability', 'year_avg_profitability', 'PROCESS__original_language'], axis=1)
drop_cols = [c for c in df_raw.columns if 'META' not in c and ('avg' in c or 'experience' in c or 'movies_before' in c)]
df_raw = df_raw.drop(drop_cols, axis=1)
df_raw = df_raw.drop(df_raw[df_raw['META__id'] == 6].index)
df_raw.to_csv('savedcsv/df_raw.csv', index=False)
df_raw = pd.read_csv('savedcsv/df_raw.csv')

In [ ]:
num = df_raw[
    (df_raw.META__profitability > df_raw.META__profitability.quantile(0.01)) & 
    (df_raw.META__profitability < df_raw.META__profitability.quantile(0.99)) &
    (df_raw.META__year >= 2000) &
    (df_raw.META__year <= 2018) &
    (df_raw.budget >= 250000)
].shape[0]

print(num)
print(num/df_raw.shape[0])

In [6]:
def get_real_columns(name_prefix, name_suffix, num_columns):
    return [f'{name_prefix}{j}{name_suffix}' for j in range(1, num_columns+1)]

def get_date_from_movie(movie):
    return date(movie['META__year'], movie['META__month'], movie['META__day'])


def get_info_df_movies_per_value_range_columns(df, columns):
    result = {}
    for i, row in df.iterrows():
        for col in columns:
            value = row[col]
            if not pd.isna(value):
                if value in list(result.keys()):
                    result[value]['movies'] = result[value]['movies'].append(df.iloc[[i]])
                else:
                    result[value] = {'movies': df.iloc[[i]]}
    return result

# def get_info_df_movies_per_value_range_columns(df, columns):
#     def rows_with_value(value):
#         return df[df.apply(lambda row: value in [row[column] for column in columns], axis=1)]

#     unique_values = list(set([movie[column] for i, movie in df.iterrows() for column in columns]))
#     return {value: rows_with_value(value) for value in unique_values}

def calculate_info_per_previous_movies_per_value(df, 
                                                 info_dict, 
                                                 calc_budget=False, 
                                                 calc_movies_before=False, 
                                                 calc_experience=False, 
                                                 calc_revenue=False,
                                                ):
    
    def get_average_profit(df):
        return (df['META__revenue'] - df['budget']).mean()
    
    def get_experience(df, row):
        return (row['META__year'] - df.META__year.min())
    
    def is_movie_before(movie, compare_date):
        return date(movie.META__year, movie.META__month, movie.META__day) < compare_date

    for value_name in info_dict:
        value = info_dict[value_name]
        value['avg_profit'] = {}
        if calc_budget:
            value['avg_budget'] = {}
        if calc_experience:
            value['experience'] = {}
        if calc_revenue:
            value['avg_revenue'] = {}
        if calc_movies_before:
            value['movies_before'] = {}

        movies = value['movies']
        for movie_index, movie in movies.iterrows():
            current_date = get_date_from_movie(movie)
            str_date = str(current_date)
            if calc_experience and str_date not in list(value['experience'].keys()):
                value['experience'][str_date] = get_experience(movies, movie)
            if str_date not in list(value['avg_profit'].keys()):
                movies_before = movies[movies.apply(lambda row: is_movie_before(row, current_date), axis=1)]
                if not movies_before.empty:
                    value['avg_profit'][str_date] = get_average_profit(movies_before)
                    if calc_movies_before:
                        value['movies_before'][str_date] = movies_before.shape[0]
                    if calc_revenue:
                        value['avg_revenue'][str_date] = movies_before['META__revenue'].mean()                        
                    if calc_budget:
                        value['avg_budget'][str_date] = movies_before['budget'].mean()
                    

def get_columns_from_info_dict(df, info_dict, real_columns, new_column_prefix='', nested_info=False):
    def get_first_key(d):
        return d[list(d.keys())[0]]
    
    new_columns_data = {}
    info_names = [key for key in get_first_key(info_dict).keys() if key != 'movies']
    print(info_names)
    for index, real_column in enumerate(real_columns):
        print(real_column)
        new_column_names = [f'{new_column_prefix}_{key}' if len(real_columns) == 1 else f'{new_column_prefix}_{index+1}_{key}' for key in info_names] 
        print(new_column_names)
            
        for new_column_name in new_column_names:
            new_columns_data[new_column_name] = []

        for movie_index, movie in df.iterrows():
            value = movie[real_column]
            str_date = str(get_date_from_movie(movie))
            if pd.isna(value):
                for col in new_column_names:
                    new_columns_data[col].append(np.nan)
            else:
                for name_index, new_column_name in enumerate(new_column_names):
                    year_info = info_dict[value][info_names[name_index]].get(str_date, np.nan)
                    new_columns_data[new_column_name].append(year_info)
    return new_columns_data

In [9]:
def add_features(df, 
                 columns,
                 new_column_prefix,
                 calc_movies_before=False, 
                 calc_budget=False, 
                 calc_experience=False,
                 calc_revenue=False):
    print('obtaine movies')
    
    t = time.process_time()
    data_info = get_info_df_movies_per_value_range_columns(df, columns)
    print(f'time: {time.process_time() - t}')
    
    data_info_copy = data_info.copy()
    print('calculate info')
    t = time.process_time()
    calculate_info_per_previous_movies_per_value(df, 
                                                 data_info_copy, 
                                                 calc_movies_before=calc_movies_before, 
                                                 calc_budget=calc_budget, 
                                                 calc_experience=calc_experience,
                                                 calc_revenue=calc_revenue)
    print(f'time: {time.process_time() - t}')
    
    print('get columns')
    t = time.process_time()
    feature_columns = get_columns_from_info_dict(df, data_info_copy, columns, new_column_prefix=new_column_prefix)
    print(f'time: {time.process_time() - t}')
    for column in feature_columns.keys():
        df[column] = feature_columns[column]
    return df


def add_companies_features(df):
    columns = get_real_columns('META__production_company_', '', 3)
    return add_features(df, columns, 'production_company', calc_revenue=True)


def add_cast_features(df):
    columns = get_real_columns('META__cast_', '_name', 8)
    return add_features(df, columns, 'cast', calc_movies_before=True, calc_revenue=True, calc_experience=True)


def add_crew_features(df):
    for column in [c for c in list(df.columns) if 'META__crew' in c]:
        df = add_features(df, [column], column.replace('META__', ''), calc_movies_before=True, calc_revenue=True)
    return df


def add_collection_features(df):
    return add_features(df, ['META__collection_name'], 'collection', calc_revenue=True)


def get_avg_cast_info(df):
    def get_average_cast_movies(row, columns):
        avgs = [row[col] for col in columns if not pd.isna(row[col])]
        return mean(avgs) if len(avgs) else np.nan
    
    def get_cast_average(df, column_name):
        columns = [col for col in df.columns if 'cast_' in col and column_name in col and not 'cast_avg' in col]
        return [get_average_cast_movies(row, columns) for i, row in df.iterrows()]
    
    for column in ['revenue', 'profit', 'experience', 'movies_before']:
        df[f'cast_avg_{column}'] = get_cast_average(df, column)
    return df


def prepare_df(df_original):
    overall_time = time.process_time()
    df = df_original.copy()
    df = add_companies_features(df)
    df = add_cast_features(df)
    df = add_crew_features(df)
    df = add_collection_features(df)
    df = get_avg_cast_info(df)
    df['year_avg_revenue'] = calc_smooth_mean(df, by='META__year', on='META__revenue', m=5)
    print(f'overall time: {time.process_time() - overall_time}')
    return df
    

In [8]:
df_all = prepare_df(df_raw)

obtaine movies
time: 49.276002803
calculate info
time: 70.130137781
get columns
['avg_profit', 'avg_revenue']
META__production_company_1
['production_company_1_avg_profit', 'production_company_1_avg_revenue']
META__production_company_2
['production_company_2_avg_profit', 'production_company_2_avg_revenue']
META__production_company_3
['production_company_3_avg_profit', 'production_company_3_avg_revenue']
time: 2.9208630349999964
obtaine movies
time: 165.15474833499997
calculate info
time: 123.57350818999998
get columns
['avg_profit', 'experience', 'avg_revenue', 'movies_before']
META__cast_1_name
['cast_1_avg_profit', 'cast_1_experience', 'cast_1_avg_revenue', 'cast_1_movies_before']
META__cast_2_name
['cast_2_avg_profit', 'cast_2_experience', 'cast_2_avg_revenue', 'cast_2_movies_before']
META__cast_3_name
['cast_3_avg_profit', 'cast_3_experience', 'cast_3_avg_revenue', 'cast_3_movies_before']
META__cast_4_name
['cast_4_avg_profit', 'cast_4_experience', 'cast_4_avg_revenue', 'cast_4_mov

time: 0.9954649649999965
obtaine movies
time: 4.795272720999947
calculate info
time: 3.2316348000000517
get columns
['avg_profit', 'avg_revenue', 'movies_before']
META__crew__art__property_master
['crew__art__property_master_avg_profit', 'crew__art__property_master_avg_revenue', 'crew__art__property_master_movies_before']
time: 0.98768953900003
obtaine movies
time: 10.69515894999995
calculate info
time: 7.516860092999991
get columns
['avg_profit', 'avg_revenue', 'movies_before']
META__crew__art__set_decoration
['crew__art__set_decoration_avg_profit', 'crew__art__set_decoration_avg_revenue', 'crew__art__set_decoration_movies_before']
time: 1.0221063320000212
obtaine movies
time: 4.429058052999949
calculate info
time: 3.176390837999975
get columns
['avg_profit', 'avg_revenue', 'movies_before']
META__crew__visualeffects__visual_effects_supervisor
['crew__visualeffects__visual_effects_supervisor_avg_profit', 'crew__visualeffects__visual_effects_supervisor_avg_revenue', 'crew__visualeffects

[2200000.0, 3624000.0, 1822000.0, 7000000.0]
[2557544.0, 16554699.0, 68091343.0, 27623538.444444444, 35902679.6]
[188644470.5, 11137801.0, 18098903.666666668, 18995334.0, 16116878.0]
[45774043.666666664, 12348905.0]
[]
[172252799.0, 30000000.0, 939423.0]
[42323252.25, 158746968.33333334, 108302700.8, 185734218.0, 70335860.72727273, 40675149.875]
[40629135.0, 75211325.0, 82281358.0, 463000000.0]
[113160101.6, 194725767.83333334, 66623502.2, 114818412.57142857, 160937519.57142857, 16491080.0, 59130120.14285714]
[14884440.25, 118936126.8, 14286559.0, 74909426.33333333, 20000000.0]
[149327680.6470588, 42649774.55555555, 6684119.333333333, 8123860.0, 7993039.0, 17626234.0, 106683817.0]
[60305013.54545455, 42269945.75, 63384861.0, 1050861.0, 156167015.0]
[132603281.33333333, 50700145.5, 28797153.14285714, 22915124.5, 4185250.0, 1189315.0]
[100837156.0, 2945600.0]
[62003534.18181818, 13121611.0, 126218959.5, 129185492.33333333, 123306800.5]
[142030608.13333333, 189833357.0, 5200000.0, 3576999

[77451161.33333333, 85061317.0, 89732411.2, 61534877.75, 73575115.0, 38745994.5]
[158597991.08333334, 38619405.0, 23144370.5, 31171274.0, 31533564.5, 17410017.666666668, 48268122.0]
[7292175.0, 31150949.0, 40227006.0, 235483004.0, 97387965.0]
[278019771.0, 113440556.33333333, 85205531.5, 267972.0, 54682547.0, 78795308.78571428]
[16026105.263157895, 7233000.0, 8121200.0, 3400000.0, 100000.0, 36345728.2]
[17164470.588235293, 10735333.333333334, 56000000.0, 7424255.857142857]
[134778645.5, 136000000.0, 161834276.0, 49563531.166666664, 6755271.0, 34199907.11111111]
[136638464.2857143, 76109471.42857143, 60073811.5, 118792233.8235294, 63014295.93333333, 54754247.625, 139687825.75, 14063848.0]
[113714244.0, 252909177.0, 53319001.61904762, 51250030.0, 135898662.5, 139372542.875, 30670565.333333332]
[146739044.625, 154404294.45454547, 32683225.5, 66872263.333333336, 94639340.25, 51338693.5, 58254463.1, 83789899.57142857]
[347454023.1111111, 178604206.8, 19306279.0, 9576495.0, 32403923.77777777

[33716959.0, 55556768.25, 25196249.0, 36492169.333333336]
[173517462.8, 30417597.333333332, 9771658.0, 9247881.0]
[59518430.333333336, 120162823.0, 81480709.0, 87807261.1111111, 48362460.0, 67518935.8, 40776037.09090909, 53394821.5]
[26589000.0, 69995385.0, 35000000.0]
[214425469.57142857, 41555634.0, 3049135.0, 16656125.0]
[31480353.666666668, 19204929.0, 60921760.5, 285444603.0, 63513743.0, 51702483.0]
[30225995.333333332, 352194034.0, 272742922.0, 25215865.4, 64521665.666666664, 227853986.0, 31596911.0]
[]
[]
[2030075.0, 99867853.5, 12771289.5, 47152853.571428575, 13736767.6, 8720000.0, 1382269.5, 11423631.909090908]
[147759585.75, 75964578.41666667, 197179218.66666666, 9750000.0, 150781537.33333334, 51407410.625, 87102908.375, 83905402.0]
[30941609.2, 21107187.2, 51689961.75, 369448396.8333333, 93751038.0, 52629912.615384616, 25251638.0, 181698530.0]
[61551143.0, 21592501.636363637, 10311845.5]
[42776760.0, 20483423.0, 42776760.0]
[171049903.57142857, 135505361.26666668, 87349007.2

...

[11.0, 2.0]
[]
[38.0, 4.0]
[5.0, 1.0, 1.0, 1.0]
[2.0, 1.0]
[40.0, 44.0, 12.0, 1.0]
[1.0, 1.0, 3.0, 1.0]
[20.0, 6.0, 1.0, 3.0, 1.0, 2.0]
[55.0, 25.0, 7.0, 13.0, 2.0, 3.0]
[25.0, 13.0, 34.0, 19.0, 7.0, 8.0, 11.0, 4.0]
[23.0, 3.0, 20.0, 7.0]
[24.0, 3.0, 16.0, 18.0, 1.0, 1.0]
[16.0, 24.0, 34.0, 10.0, 3.0, 7.0, 3.0, 1.0]
[1.0, 7.0, 6.0]
[1.0, 2.0]
[3.0, 1.0, 1.0]
[33.0, 5.0, 3.0, 6.0, 10.0, 19.0]
[19.0, 21.0, 1.0]
[2.0, 1.0, 1.0, 3.0]
[4.0, 4.0, 12.0, 3.0]
[4.0, 4.0, 1.0, 1.0, 7.0, 1.0]
[]
[17.0, 6.0, 17.0, 9.0, 5.0, 5.0]
[20.0, 14.0, 9.0, 6.0, 2.0, 1.0, 15.0]
[54.0, 32.0, 17.0, 19.0, 12.0, 4.0, 2.0, 1.0]
[5.0, 10.0, 1.0, 2.0]
[]
[1.0, 10.0, 8.0, 4.0, 2.0, 1.0]
[66.0, 31.0, 13.0, 8.0, 8.0, 27.0, 19.0, 15.0]
[29.0, 37.0, 29.0, 13.0, 3.0, 2.0]
[43.0, 16.0, 18.0, 20.0, 1.0]
[12.0, 7.0, 25.0, 33.0, 15.0, 9.0, 2.0, 6.0]
[19.0, 21.0, 19.0, 1.0, 1.0]
[4.0, 5.0, 1.0, 7.0, 2.0, 6.0]
[1.0, 5.0, 5.0, 10.0, 6.0, 13.0, 32.0]
[10.0, 2.0, 6.0, 1.0, 2.0, 12.0]
[5.0, 22.0, 10.0, 16.0, 3.0, 9.0, 2.0, 5.0]
[1

[17.0, 15.0, 7.0, 10.0, 2.0, 5.0]
[7.0, 4.0, 10.0, 12.0, 16.0, 13.0]
[1.0, 2.0, 10.0, 1.0, 2.0]
[4.0, 7.0, 1.0, 2.0, 8.0]
[7.0, 3.0]
[1.0, 2.0, 3.0, 3.0, 2.0, 1.0, 2.0]
[1.0]
[2.0, 1.0, 2.0]
[6.0, 3.0, 2.0]
[7.0, 1.0, 1.0, 14.0, 17.0, 6.0, 16.0]
[12.0, 6.0, 2.0, 4.0, 3.0, 4.0]
[8.0, 2.0, 7.0, 2.0, 7.0, 2.0, 1.0]
[1.0, 1.0, 4.0, 1.0]
[12.0, 12.0, 7.0, 1.0, 9.0, 13.0, 13.0, 9.0]
[3.0, 3.0, 1.0]
[2.0, 14.0, 6.0, 2.0, 13.0, 10.0, 1.0]
[1.0, 3.0, 1.0]
[]
[21.0, 6.0, 6.0, 1.0, 3.0]
[]
[3.0, 2.0, 4.0, 5.0]
[]
[1.0, 2.0, 21.0, 2.0]
[1.0, 2.0, 1.0, 2.0, 1.0, 2.0]
[2.0, 1.0, 1.0]
[1.0, 1.0, 1.0]
[5.0, 1.0, 3.0]
[1.0, 1.0, 2.0, 1.0, 1.0]
[]
[1.0]
[2.0, 1.0, 2.0]
[3.0, 1.0, 1.0, 1.0]
[2.0, 2.0, 3.0, 1.0, 1.0, 1.0, 1.0]
[6.0, 12.0, 2.0, 2.0, 1.0]
[3.0, 1.0]
[2.0, 3.0, 1.0]
[3.0, 2.0, 1.0]
[]
[]
[4.0, 2.0]
[]
[6.0, 3.0, 5.0, 3.0, 1.0]
[3.0, 17.0, 1.0, 9.0, 10.0]
[4.0]
[22.0, 13.0, 15.0, 1.0, 11.0, 27.0, 8.0, 3.0]
[6.0, 1.0, 4.0, 1.0, 1.0, 1.0, 1.0]
[4.0, 1.0, 3.0, 4.0, 1.0, 1.0]
[1.0, 3.0, 1.0]
[3.0

[10.0, 9.0, 12.0, 1.0, 1.0, 2.0, 1.0, 1.0]
[50.0, 5.0, 10.0, 2.0, 1.0, 1.0, 1.0, 3.0]
[2.0, 1.0, 1.0, 2.0, 1.0, 3.0]
[20.0, 14.0, 2.0]
[7.0, 31.0, 4.0, 4.0, 11.0, 12.0, 6.0, 8.0]
[]
[5.0, 2.0]
[]
[3.0, 1.0, 2.0]
[]
[]
[2.0, 3.0, 1.0, 1.0]
[5.0, 5.0, 1.0, 1.0, 1.0]
[]
[]
[15.0, 5.0, 10.0, 29.0, 2.0, 2.0, 13.0, 1.0]
[]
[14.0, 1.0, 1.0]
[7.0, 3.0, 2.0, 1.0, 2.0]
[9.0, 9.0, 12.0, 7.0, 21.0, 11.0, 1.0]
[3.0, 2.0, 2.0, 11.0, 1.0, 34.0, 1.0]
[19.0, 14.0, 15.0, 35.0, 7.0, 10.0, 36.0, 1.0]
[32.0, 14.0, 16.0, 2.0, 3.0, 5.0, 2.0]
[9.0, 2.0, 8.0, 1.0, 2.0, 1.0]
[1.0]
[2.0, 4.0, 3.0]
[8.0, 6.0, 1.0, 45.0, 27.0, 7.0, 5.0, 8.0]
[28.0, 25.0, 1.0, 7.0, 6.0, 1.0, 11.0]
[41.0, 18.0, 19.0, 14.0, 5.0, 1.0, 27.0, 20.0]
[11.0, 14.0, 11.0, 1.0, 17.0, 12.0]
[33.0, 16.0, 10.0, 12.0]
[3.0, 1.0, 1.0, 3.0]
[9.0, 11.0, 9.0, 7.0, 26.0, 2.0, 7.0]
[16.0, 23.0, 24.0, 6.0, 15.0, 6.0, 1.0, 3.0]
[5.0, 28.0, 3.0, 10.0, 11.0, 24.0, 2.0]
[5.0, 6.0, 12.0, 9.0, 13.0, 5.0, 5.0]
[14.0, 1.0, 11.0, 23.0, 2.0]
[26.0, 17.0, 6.0, 13.

[4.0, 26.0, 11.0, 6.0, 7.0]
[]
[3.0, 1.0, 1.0, 1.0, 2.0]
[4.0, 3.0, 1.0, 1.0, 2.0]
[]
[6.0, 8.0, 3.0, 9.0, 2.0, 6.0, 4.0]
[1.0, 2.0, 1.0, 1.0]
[26.0, 5.0, 8.0, 4.0, 18.0, 5.0, 8.0, 12.0]
[2.0]
[]
[2.0]
[1.0, 1.0, 1.0, 1.0]
[1.0, 1.0]
[4.0, 2.0, 1.0]
[7.0, 1.0, 3.0]
[]
[4.0, 4.0, 1.0, 4.0, 2.0]
[1.0]
[6.0, 4.0, 6.0, 1.0]
[1.0, 1.0]
[3.0, 3.0, 1.0, 4.0, 5.0]
[26.0, 9.0, 22.0, 13.0, 5.0, 17.0]
[5.0, 1.0, 2.0, 1.0]
[29.0, 21.0, 14.0, 3.0, 3.0, 1.0, 4.0]
[1.0, 1.0, 2.0, 2.0]
[4.0, 18.0, 5.0, 1.0, 11.0, 10.0]
[4.0, 35.0, 2.0, 14.0, 12.0, 26.0, 1.0]
[19.0, 26.0, 1.0, 1.0, 6.0]
[3.0, 6.0, 1.0]
[28.0, 8.0, 5.0, 18.0, 17.0, 15.0, 3.0, 18.0]
[1.0]
[2.0, 31.0, 31.0, 6.0, 10.0, 24.0, 5.0]
[7.0, 2.0, 3.0, 2.0, 4.0, 9.0, 2.0]
[6.0, 27.0, 22.0, 9.0, 9.0, 7.0, 4.0]
[6.0]
[1.0, 2.0]
[7.0, 32.0, 21.0, 10.0, 15.0, 2.0]
[1.0, 1.0, 4.0]
[7.0, 5.0, 2.0, 6.0]
[1.0, 1.0, 5.0, 2.0, 1.0]
[6.0, 1.0, 5.0, 6.0, 1.0]
[1.0]
[17.0, 4.0, 1.0, 2.0, 15.0, 4.0, 1.0]
[22.0, 4.0, 19.0, 30.0, 8.0, 10.0, 5.0, 7.0]
[5.0, 1.0, 

[1.0, 2.0, 1.0, 3.0, 1.0]
[10.0, 8.0, 7.0, 20.0, 7.0, 1.0]
[7.0, 13.0, 9.0, 3.0, 5.0, 6.0, 4.0]
[25.0, 17.0, 17.0, 22.0, 6.0, 25.0]
[27.0, 38.0, 6.0, 1.0, 2.0, 3.0]
[1.0]
[1.0, 2.0, 8.0, 2.0, 4.0]
[1.0]
[]
[2.0, 3.0, 1.0, 2.0]
[3.0, 1.0, 1.0]
[3.0, 18.0, 25.0, 12.0, 2.0, 1.0]
[13.0, 1.0, 4.0, 1.0]
[4.0, 25.0, 12.0, 1.0, 4.0, 5.0, 12.0]
[1.0]
[19.0, 38.0]
[3.0]
[17.0, 2.0]
[14.0, 25.0, 5.0, 6.0, 7.0, 3.0, 14.0]
[]
[1.0]
[2.0, 5.0, 1.0, 5.0, 1.0]
[12.0, 16.0, 14.0, 16.0, 16.0, 12.0, 1.0]
[7.0, 1.0, 1.0, 35.0, 9.0]
[13.0, 7.0, 4.0, 3.0, 2.0]
[14.0, 16.0, 5.0, 3.0, 1.0, 3.0]
[27.0, 23.0, 7.0, 12.0, 28.0, 11.0, 10.0]
[1.0, 9.0]
[5.0, 6.0, 1.0]
[18.0, 10.0, 10.0, 1.0, 2.0, 1.0]
[11.0, 1.0, 2.0, 13.0, 6.0, 19.0, 31.0, 9.0]
[2.0]
[24.0, 2.0, 2.0, 1.0]
[]
[16.0, 1.0, 6.0, 1.0, 2.0, 1.0, 8.0]
[1.0]
[33.0, 1.0, 2.0, 2.0]
[4.0, 12.0, 4.0, 6.0, 2.0, 1.0, 10.0]
[6.0, 8.0, 19.0, 14.0, 3.0, 11.0, 4.0, 2.0]
[1.0, 1.0, 11.0, 4.0, 1.0, 4.0]
[27.0, 27.0, 1.0, 3.0, 2.0]
[7.0, 4.0, 1.0, 2.0, 2.0]
[3.0, 1.0,

In [46]:
df_all.to_csv('savedcsv/dataset_all.csv', index=False)

In [47]:
df_all = pd.read_csv('savedcsv/dataset_all.csv')